In [4]:
import pandas as pd


In [6]:
pip install pandahouse

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandahouse: filename=pandahouse-0.2.7-py2.py3-none-any.whl size=5912 sha256=b57732fe392f689b2a8ca287484e836fe9c46037b4ba9b06c6666d8154e3a4c9
  Stored in directory: c:\users\domodedovo\appdata\local\pip\cache\wheels\2b\75\4a\22824efa98a925fce4a1f2c18e85b55eb75179dfb9e9d59f45
Successfully built pandahouse
Note: you may need to restart the kernel to use updated packages.


In [7]:

import pandahouse as ph

In [8]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses/',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [9]:
# создаем запрос, db автоматически заменится на значение из database
q = '''
    SELECT
count(st_id) as hard_students
FROM
    (
    SELECT
        st_id,
        toStartOfMonth(timest) as time_month,
        sum(correct) as sum_sucsess
    FROM
    default.peas
    GROUP BY
    st_id, time_month
    HAVING
    sum_sucsess >= 20
    )
    '''
# отправляем запрос и записываем результат в пандасовский датафрейм
q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test

,hard_students
0,136


In [ ]:
2.1.2 Задача

In [10]:
# с комментариями в юпитере почему то не работает запрос , но в Табиксе все работает я проверял
r = '''
  SELECT t1.test_grp as test_grp, t1.ARPU as ARPU, t2.ARPPU as ARPPU, t3.CR as CR, t4.CR_Activ as CR_Activ,
  t5.CR_Activ_math as CR_Activ_math --выводим требуемые заданием данные, групируя по группам test_grp
FROM(
    SELECT test_grp, round(sum(money)/uniqExact(st_id), 1) as ARPU --создаем ARPU
    FROM 
    (
        SELECT --Соединяем 3 таблицы через full JOIN чтобы учитывать данные каждых таблиц,
               --по идее можно сделать вьюху этого запроса так как его часто использую
            t1.st_id as st_id, t1.timest as timest, t3.sale_time as sale_time, t1.correct as correct,
            t1.subject as subject, t2.test_grp as test_grp, t3.money as money
                FROM
                default.peas AS t1
            full JOIN
                default.studs AS t2 ON t1.st_id = t2.st_id
            full JOIN
                default.final_project_check AS t3 ON t1.st_id = t3.st_id
    )
    GROUP BY test_grp
    ) as t1
JOIN  --Объединяем ARPU и ARPРU
    (
    SELECT --создаем ARPPU
test_grp, round(sum(sum_money)/uniqExact(st_id), 1) as ARPPU
FROM
    (
        SELECT  st_id, test_grp, sum(money) as sum_money --создаем подзапрос для того чтобы посчитать общую сумму каждого
                                                         --пользователя фильтруя по суме !=0
        FROM
            (
            SELECT 
            t1.st_id as st_id, t1.timest as timest, t3.sale_time as sale_time, t1.correct as correct,
            t1.subject as subject, t2.test_grp as test_grp, t3.money as money
                FROM
                default.peas AS t1
            full JOIN
                default.studs AS t2 ON t1.st_id = t2.st_id
            full JOIN
                default.final_project_check AS t3 ON t1.st_id = t3.st_id
          
            )
        GROUP BY st_id, test_grp
    )
    WHERE sum_money != 0
    GROUP BY test_grp
    ) as t2 on t1.test_grp = t2.test_grp
JOIN --Объединяем ARPU и ARPРU и  CR
    (
        SELECT
test_grp, round((COUNTIf(sum_money > 0)*100)/count(*), 1) as CR --Считаем  CR через COUNTIf
FROM
    (
        SELECT  st_id, test_grp, sum(money) as sum_money --создаем подзапрос для того чтобы посчитать общую сумму каждого
                                                         --пользователя без фильтрации по суме !=0 так как нам нужны все клиенты
        FROM
            (
            SELECT 
            t1.st_id as st_id, t1.timest as timest, t3.sale_time as sale_time, t1.correct as correct, 
            t1.subject as subject, t2.test_grp as test_grp, t3.money as money
                FROM
                default.peas AS t1
            full JOIN
                default.studs AS t2 ON t1.st_id = t2.st_id
            full JOIN
                default.final_project_check AS t3 ON t1.st_id = t3.st_id
          
            )
        GROUP BY st_id, test_grp
    )
    GROUP BY test_grp
    ) AS t3 ON t1.test_grp = t3.test_grp
JOIN --Объединяем ARPU и ARPРU и  CR и CR_Activ
    (
    SELECT
test_grp, round((COUNTIf(sum_money > 0 and sum_sucsess > 10)*100)/count(*), 1) as CR_Activ --активный пользователь считается тем
                                                                --кто выполнил больше 10 заданий
FROM
    (
        SELECT  st_id, test_grp, sum(correct) as sum_sucsess, sum(money) as sum_money -- подзапрос где в дополнении 
                                                   --к предыдущему мы добовляем количество успешных выполненых работ
        FROM
            (
            SELECT 
            t1.st_id as st_id, t1.timest as timest, t3.sale_time as sale_time, t1.correct as correct, 
            t1.subject as subject, t2.test_grp as test_grp, t3.money as money
                FROM
                default.peas AS t1
            full JOIN
                default.studs AS t2 ON t1.st_id = t2.st_id
            full JOIN
                default.final_project_check AS t3 ON t1.st_id = t3.st_id
          
            )
        GROUP BY st_id, test_grp
    )
    GROUP BY test_grp
    ) AS t4 ON t1.test_grp = t4.test_grp
JOIN --Объединяем ARPU и ARPРU и  CR и CR_Activ и CR_Activ_math
    (
    SELECT
test_grp, round((COUNTIf(sum_money > 0 and sum_sucsess >= 2 and subject = 'Math')*100)/uniqExact(st_id), 1) as CR_Activ_math
                                   -- находим CR тех кто решил 2 и больше задачи по математике
FROM
    (
        SELECT  st_id, test_grp, sum(correct) as sum_sucsess, subject, sum(money) as sum_money --добавляем колонку subject
        FROM
            (
            SELECT 
            t1.st_id as st_id, t1.timest as timest, t3.sale_time as sale_time, t1.correct as correct, 
            t1.subject as subject, t2.test_grp as test_grp, t3.money as money
                FROM
                default.peas AS t1
            full JOIN
                default.studs AS t2 ON t1.st_id = t2.st_id
            full JOIN
                default.final_project_check AS t3 ON t1.st_id = t3.st_id
          
            )
        GROUP BY st_id, test_grp, subject
    )
    GROUP BY test_grp
    ) AS t5 ON t1.test_grp = t5.test_grp
WHERE test_grp = 'control' or test_grp = 'pilot' -- удаляем какую то пустую строку
    '''
# отправляем запрос и записываем результат в пандасовский датафрейм
# с комментариями в юпитере почему то не работает запрос , но в Табиксе все работает я проверял
q_test_2 = ph.read_clickhouse(query=r, connection=connection_default)
q_test_2

ClickhouseException: b'<html>\r\n<head><title>414 Request-URI Too Large</title></head>\r\n<body>\r\n<center><h1>414 Request-URI Too Large</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'